<a href="https://colab.research.google.com/github/Serena-Wang/Anagrams/blob/master/tensorflow_probability_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_probability as tfp

TensorFlow 2.x selected.


In [0]:
# Suppose `SomeKernel` acts on vectors (rank-1 tensors)
scalar_kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=0.5,
    length_scale=1.2)
scalar_kernel.batch_shape
# ==> []

# `x` and `y` are batches of five 2-D vectors:
np.random.seed(84963)
x = np.random.standard_normal((5, 2))
y = np.random.standard_normal((5, 2))
scalar_kernel.apply(x, y)#.shape
# ==> [5]


<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.17942807, 0.10982524, 0.10400961, 0.0548934 , 0.18378305],
      dtype=float32)>

In [0]:
batch_kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=[.5, 0.2],
    length_scale=[1.2, 1.7])

batch_kernel.batch_shape
# ==> []

# `x` and `y` are batches of five 2-D vectors:
np.random.seed(84963)
x = np.random.standard_normal((5, 2))
y = np.random.standard_normal((5, 2))
batch_kernel.apply(x, y, example_ndims=1)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.17942807, 0.10982524, 0.10400961, 0.0548934 , 0.18378305],
       [0.03390661, 0.02654967, 0.02583959, 0.0187927 , 0.0343142 ]],
      dtype=float32)>

In [0]:
np.random.seed(84963)
x = np.concatenate(
  (np.array([0, 0, 0, 1, 1]).reshape((5, 1)), np.linspace(1, 5, 5).reshape((5, 1))),
  axis = 1
)
y = np.concatenate(
  (np.array([0, 1, 1, 1, 2]).reshape((5, 1)), np.linspace(1, 2, 5).reshape((5, 1))),
  axis = 1
)

print(x)
print(y)

[[0. 1.]
 [0. 2.]
 [0. 3.]
 [1. 4.]
 [1. 5.]]
[[0.   1.  ]
 [1.   1.25]
 [1.   1.5 ]
 [1.   1.75]
 [2.   2.  ]]


In [0]:
k = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=0.5,
    length_scale=1.2)

k_s = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=0.7,
    length_scale=1.7)

result = k.apply(x[:,1:], y[:,1:], example_ndims=1)
result_s = k_s.apply(x[:,1:], y[:,1:], example_ndims=1)
inds = tf.where(x[:, 0] == y[:, 0])#[0].tolist()

print(result.get_shape())

(5,)


In [0]:
shape = result.get_shape()  # Should get this from result shape

delta = tf.SparseTensor(inds, tf.gather_nd(result_s, inds), shape)
print(tf.sparse.to_dense(delta))
final_result = result + tf.sparse.to_dense(delta)
print(final_result)

tf.Tensor([0.48999998 0.         0.         0.20408575 0.        ], shape=(5,), dtype=float32)
tf.Tensor([0.74       0.20564438 0.11445834 0.24719116 0.01098423], shape=(5,), dtype=float32)


In [0]:
k = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=[0.5, 0.7],
    length_scale=[1.2, 2.4])

k_s = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=[0.7, 1.2],
    length_scale=[1.7, 1.3])

result = k.apply(x[:,1:], y[:,1:], example_ndims=1)
result_s = k_s.apply(x[:,1:], y[:,1:], example_ndims=1)
inds = tf.where(x[:, 0] == y[:, 0])#[0].tolist()

print(result.get_shape())

(2, 5)


In [0]:
shape = result.get_shape()  # Should get this from result shape

delta = tf.SparseTensor(inds, tf.gather_nd(result_s, inds), shape)
print(tf.sparse.to_dense(delta))
final_result = result + tf.sparse.to_dense(delta)
print(final_result)

In [0]:
k = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=[[0.5, 0.7], [0.2, 0.3]],
    length_scale=[1.2, 2.4])

k_s = tfp.math.psd_kernels.ExponentiatedQuadratic(
    amplitude=[0.7, 1.2],
    length_scale=[1.7, 1.3])

result = k.apply(x[:,1:], y[:,1:], example_ndims=1)
result_s = k_s.apply(x[:,1:], y[:,1:], example_ndims=1)
inds = tf.where(x[:, 0] == y[:, 0])#[0].tolist()

print(result.get_shape())
print(result_s.get_shape())

(2, 2, 5)
(2, 5)


In [0]:
inds.numpy().squeeze().tolist()

[0, 3]

In [0]:
result[:,:,0]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.25      , 0.48999998],
       [0.04      , 0.09      ]], dtype=float32)>

In [0]:
tf.gather_nd(result, inds[tf.newaxis, tf.newaxis, :, 0], batch_dims=2)

InvalidArgumentError: ignored

In [0]:
result

<tf.Tensor: shape=(2, 2, 5), dtype=float32, numpy=
array([[[0.25      , 0.2056444 , 0.11445834, 0.0431054 , 0.01098423],
        [0.48999998, 0.46664894, 0.403063  , 0.31575045, 0.22433837]],

       [[0.04      , 0.0329031 , 0.01831333, 0.00689686, 0.00175748],
        [0.09      , 0.08571103, 0.07403198, 0.05799498, 0.041205  ]]],
      dtype=float32)>

In [0]:
inds[tf.newaxis, tf.newaxis, :, 0]

<tf.Tensor: shape=(1, 1, 2), dtype=int64, numpy=array([[[0, 3]]])>

In [0]:
inds[:,0]

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 3])>

In [56]:
indices = [[tf.newaxis,tf.newaxis,0],[tf.newaxis,tf.newaxis,3]]
tf.gather_nd(result,indices)

ValueError: ignored